In [5]:
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = "notebook_connected"

class Traccia:
    def __init__(self):
        self.x = []
        self.y = []
        self.z = []
        self.dedx = []
        self.tipo = ""
        self.mc = False

def parse_slices(filename):
    all_slices = []
    current_slice = {}
    vertice = None
    with open(filename) as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            if "#new slice" in line:
                if current_slice:
                    all_slices.append((current_slice, vertice))
                current_slice = {}
                vertice = None
            elif line.startswith("#"):
                continue
            else:
                parts = line.split()
                if parts[0] == "VERTEX":
                    vert_x, vert_y, vert_z = map(float, parts[1:4])
                    vertice = (vert_x, vert_y, vert_z)
                else:
                    try:
                        dedx = float(parts[3])
                        x, y, z = map(float, parts[0:3])
                        nome_traccia = parts[4]
                        tipo = parts[5]
                        extra = parts[6] if len(parts) > 6 else ""
                    except ValueError:
                        x, y, z = map(float, parts[0:3])
                        dedx = 0.0
                        nome_traccia = parts[3]
                        tipo = parts[4]
                        extra = parts[5] if len(parts) > 5 else ""
                    mc = (extra == "mc")
                    if nome_traccia not in current_slice:
                        current_slice[nome_traccia] = Traccia()
                        current_slice[nome_traccia].tipo = tipo
                        current_slice[nome_traccia].mc = mc
                    t = current_slice[nome_traccia]
                    t.x.append(x)
                    t.y.append(y)
                    t.z.append(z)
                    t.dedx.append(dedx)
        if current_slice:
            all_slices.append((current_slice, vertice))
    return all_slices

# ---- Bordo TPC ----
def add_tpc_edges(fig, x_range, y_range, z_range, color="black", width=4, name="TPC edge"):
    x0, x1 = x_range
    y0, y1 = y_range
    z0, z1 = z_range
    vertices = [
        [x0,y0,z0],[x1,y0,z0],[x1,y1,z0],[x0,y1,z0],
        [x0,y0,z1],[x1,y0,z1],[x1,y1,z1],[x0,y1,z1]
    ]
    edges = [
        (0,1),(1,2),(2,3),(3,0),
        (4,5),(5,6),(6,7),(7,4),
        (0,4),(1,5),(2,6),(3,7)
    ]
    for i,j in edges:
        fig.add_trace(go.Scatter3d(
            x=[vertices[i][0], vertices[j][0]],
            y=[vertices[i][1], vertices[j][1]],
            z=[vertices[i][2], vertices[j][2]],
            mode="lines",
            line=dict(color=color,width=width),
            name=name,
            showlegend=False
        ))

# ---- Volumi TPC semi-trasparenti ----
def add_tpc_planes(fig, x_range, y_range, z_range, color="#C3DAFF", opacity=0.2, name="TPC volume"):
    x0, x1 = x_range
    y0, y1 = y_range
    z0, z1 = z_range
    verts = [
        [x0,y0,z0],[x1,y0,z0],[x1,y1,z0],[x0,y1,z0],
        [x0,y0,z1],[x1,y0,z1],[x1,y1,z1],[x0,y1,z1]
    ]
    i,j,k = [],[],[]
    i += [0,0]; j += [1,2]; k += [2,3]  # z0
    i += [4,4]; j += [5,6]; k += [6,7]  # z1
    i += [0,0]; j += [3,7]; k += [7,4]  # x0
    i += [1,1]; j += [2,6]; k += [6,5]  # x1
    i += [0,0]; j += [1,5]; k += [5,4]  # y0
    i += [3,3]; j += [2,6]; k += [6,7]  # y1
    fig.add_trace(go.Mesh3d(
        x=[v[0] for v in verts],
        y=[v[1] for v in verts],
        z=[v[2] for v in verts],
        i=i,j=j,k=k,
        color=color,opacity=opacity,
        name=name,
        showscale=False
    ))

# ---- Catodo ----
def add_cathode(fig, x_pos, y_range=(-181.86,134.96), z_range=(-894.95,894.95), color="gray", opacity=0.3):
    y0, y1 = y_range
    z0, z1 = z_range
    fig.add_trace(go.Mesh3d(
        x=[x_pos,x_pos,x_pos,x_pos],
        y=[y0,y0,y1,y1],
        z=[z0,z1,z1,z0],
        i=[0,0], j=[1,2], k=[2,3],
        color=color, opacity=opacity,
        name=f"Catodo {x_pos}",
        showscale=False
    ))

# ---- VISUALIZZAZIONE PRINCIPALE ----
def disegna_slice_plotly(slice_data, slice_index, dedx_range=(0,5)):
    tracce, vertice = slice_data
    fig = go.Figure()
    line_colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728",
                   "#9467bd", "#8c564b", "#e377c2", "#7f7f7f"]
    colorbar_shown = False

    # --- TRACCE ---
    for i,(nome,t) in enumerate(tracce.items()):
        line_color = line_colors[i % len(line_colors)]
        if t.mc:
            marker=dict(size=5,symbol="circle",color=[0]*len(t.x),
                        colorscale=[[0,line_color],[1,line_color]],showscale=False)
            line=dict(color=line_color,width=3,dash="dash")
            mode="lines+markers"
        else:
            marker=dict(size=5,symbol="circle",color=t.dedx,
                        colorscale="Viridis",cmin=dedx_range[0],
                        cmax=dedx_range[1],
                        showscale=not colorbar_shown,
                        colorbar=dict(title="dE/dx") if not colorbar_shown else None)
            line=dict(color=line_color,width=0)
            mode="markers"
            colorbar_shown=True

        fig.add_trace(go.Scatter3d(
            x=t.x, y=t.y, z=t.z,
            mode=mode,
            marker=marker,
            line=line,
            name=f"{nome} ({t.tipo}){' MC' if t.mc else ''}"
        ))

    # --- VERTICE ---
    if vertice:
        fig.add_trace(go.Scatter3d(
            x=[vertice[0]],y=[vertice[1]],z=[vertice[2]],
            mode="markers",
            marker=dict(size=7,color="#22F53E",symbol="circle"),
            name="Vertex"
        ))

    # --- TPC + Catodi ---
    tpc_coords = [(-358.73,-210.0), (-210.0,-61.7), (61.7,210.0), (210.0,358.73)]
    y_range = (-181.86,134.96)
    z_range = (-894.95,894.95)
    for i,(x0,x1) in enumerate(tpc_coords):
        add_tpc_planes(fig,(x0,x1),y_range,z_range,color="#C3DAFF",opacity=0.2,name=f"TPC{i+1}")
        add_tpc_edges(fig,(x0,x1),y_range,z_range,color="black",width=4)
    add_cathode(fig,-210.0)
    add_cathode(fig,210.0)

    # --- PULSANTI INTERATTIVI (Solo due) ---
    n_tracce = len(tracce)
    buttons = [
        dict(label="Solo Tracce",
             method="update",
             args=[{"visible":[True]*n_tracce + [False]*(len(fig.data)-n_tracce)}]),
        dict(label="Intero Detector",
             method="update",
             args=[{"visible":[True]*len(fig.data)}])
    ]
    fig.update_layout(
        #title=f"Slice {slice_index}",
        scene=dict(
            xaxis=dict(showbackground=False,visible=False),
            yaxis=dict(showbackground=False,visible=False),
            zaxis=dict(showbackground=False,visible=False),
            aspectmode='data'
        ),
        updatemenus=[dict(type="buttons", buttons=buttons, x=0, y=1.05)],
        showlegend=True,
        legend=dict(x=0,y=1,xanchor="left",yanchor="top"),
        margin=dict(l=0,r=0,b=0,t=40),
        paper_bgcolor='white',
        plot_bgcolor='white'
    )
    fig.show()

# === MAIN ===
if __name__=="__main__":
    filename="/storage/gpfs_data/icarus/local/users/sommaggio/simul_z/event_display/tracce3Ddedx.txt"
    slices = parse_slices(filename)
    print(f"Numero di slice trovate: {len(slices)}")
    slice_index =5
    if 0<=slice_index<len(slices):
        slice_data = slices[slice_index]
        disegna_slice_plotly(slice_data, slice_index, dedx_range=(0,10))
    else:
        print("Indice slice non valido.")


Numero di slice trovate: 164708
